In [31]:
import os, glob
import csv

In [39]:
csvf = open('abstract_narrations.csv', 'w')
writer = csv.writer(csvf)

In [40]:
startTag = "<AbstractNarration>"
endTag = "</AbstractNarration>"
emptyTag = "<AbstractNarration/>"

for filename in glob.glob('files/*.xml'):
    with open(os.path.join(os.getcwd(), filename), 'r', encoding='UTF8') as f: # open in readonly mode
        data = f.read()
        if emptyTag not in data:
            start = data.find(startTag) + len(startTag) + len("<![CDATA[") 
            end = data.find(endTag) - len("]]>")
    
            writer.writerow([data[start:end]])
        
csvf.close()
